In [1]:
import pandas as pd
import json5
import json
import re

D:\tmp\ipykernel_4944\1393937433.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
file_path = "gpt4_planning_gpt4_reward/out.json"
def read_csv_result(file_path=file_path):
    with open(file_path) as f:
        data = json.load(f)
    last_action_result_list = []
    for items in data:
        data_dic = {}
        data_dic["task_id"] = items["task_id"]
        data_dic["task_name"] = items["task_name"]
        data_dic["status"] = items["task_status"]
        data_dic["steps"] = items["step_list"][-1]["step_index"]
        data_dic["task_score"] = items["step_list"][-1]["task_score"]
        data_dic["task_score_rate"] = items["step_list"][-1]["task_score_rate"]
        data_dic["selector"] = items["step_list"][-1]["selector"]
        data_dic["action"] = items["step_list"][-1]["action"]
        data_dic["url"] = items["step_list"][-2]["url"]
        data_dic["match_result"] = items["step_list"][-1]["match_result"]
        last_action_result_list.append(data_dic)
    # result_list = sorted(data,key=lambda x: x["step_list"]["task_score_rate"],reverse=True)
    return last_action_result_list

all_data = read_csv_result()
all_data


[{'task_id': 0,
  'task_name': 'View the complexity rating for Frosthaven in boardgamegeek',
  'status': 'finished',
  'steps': 3,
  'task_score': '3 / 3',
  'task_score_rate': 1.0,
  'selector': '#mainbody > div.global-body-content-container.container-fluid > div > div.content.ng-isolate-scope > div:nth-child(2) > ng-include > div > ng-include > div > div.game-header > div.game-header-body > div.game-header-gameplay.hidden-game-header-collapsed.ng-scope > gameplay-module > div > div > ul > li:nth-child(4) > div.gameplay-item-secondary > span > button',
  'action': 'click[383]',
  'url': 'https://boardgamegeek.com/boardgame/295770/frosthaven',
  'match_result': "[{'url_included_match': 'boardgamegeek.'}, {'url_included_match': '/boardgame/295770/frosthaven'}, {'element_path_exactly_match': '.gameplay >li:nth-child(4)>.gameplay-item-secondary >span:nth-child(1)>.btn.btn-xs.btn-link '}]"},
 {'task_id': 100,
  'task_name': "Find Kevin Durant's bio in espn",
  'status': 'finished',
  'step

In [4]:
df = pd.DataFrame(all_data)
df.head(10)

,task_id,task_name,status,steps,task_score,task_score_rate,selector,action,url,match_result
0,0,View the complexity rating for Frosthaven in b...,finished,3,3 / 3,1.000000,#mainbody > div.global-body-content-container....,click[383],https://boardgamegeek.com/boardgame/295770/fro...,"[{'url_included_match': 'boardgamegeek.'}, {'u..."
1,100,Find Kevin Durant's bio in espn,finished,3,3 / 3,1.000000,,None,https://www.espn.com/nba/player/bio/_/id/3202/...,"[{'url_included_match': 'espn.'}, {'url_includ..."
2,101,Find camping tents that can fit 6 people and s...,llm_finished,7,3 / 4,0.750000,#search-page-wrapper > div.uhCxDib4luSIS7EWFm8...,"select_option[1137,Price: Low - High]",https://www.rei.com/c/tents/f/sc-6-person?sort...,"[{'url_included_match': 'rei.'}, {'url_exactly..."
3,102,Order an adult’s bottomless bubbles package fo...,limit,16,2 / 8,0.250000,#main > section > div > div > div.col-lg-8 > s...,click[17],https://www.carnival.com/shop/checkout/multi/r...,"[{'url_included_match': 'carnival.'}, {'url_in..."
4,103,"Find Airport information of Camarillo Airport,...",llm_finished,6,1 / 3,0.333333,,None,https://www.flightaware.com/,[{'url_included_match': 'flightaware.'}]
5,104,Give a like to the #1 track of the Real Time T...,llm_finished,4,2 / 4,0.500000,,None,https://www.youtube.com/watch?v=Vk5-c_v4gMU,"[{'url_included_match': 'last.fm'}, {'url_incl..."
6,105,Find the newest silver-colored Lexus SUV with ...,limit,25,1 / 20,0.050000,,None,https://www.cargurus.com/Cars/inventorylisting...,[{'url_included_match': 'cargurus.'}]
7,106,Find the score of the 2020 Super Bowl in nfl.com,llm_finished,4,2 / 4,0.500000,,None,https://www.nfl.com/scores/,"[{'url_included_match': 'nfl.'}, {'url_include..."
8,107,Check my trip for Smith and booking number X89...,limit,12,1 / 4,0.250000,#aa-content-frame > section:nth-child(2) > sec...,click[54],https://www.aa.com/i18n/customer-service/faqs/...,[{'url_included_match': 'aa.'}]
9,108,Show me the stats of the Athletic Club from sp...,llm_finished,3,4 / 5,0.800000,#fittPageContainer > div.StickyContainer > div...,click[354],https://insider.espn.com/soccer/team/stats/_/i...,"[{'url_included_match': 'espn.'}, {'url_includ..."


In [5]:
df["step_score"] = df["task_score"].apply(lambda x: float(x.split("/")[0]))

In [7]:
df["efficitent_score"] = df["step_score"] / df["steps"]

In [14]:

def get_human_alignment_score(row):
    if row["status"] == "finished":
        return 1
    elif row["status"] == "llm_finished":
        return row["task_score_rate"]
    elif row["status"] == "limit":
        return row["task_score_rate"] * 0.8

# 使用apply函数和自定义函数来创建新列
df['human_alignment_score'] = df.apply(get_human_alignment_score, axis=1)



In [15]:
df.head(5)

,task_id,task_name,status,steps,task_score,task_score_rate,selector,action,url,match_result,step_score,efficitent_score,human_alianment_score,human_alignment_score
0,0,View the complexity rating for Frosthaven in b...,finished,3,3 / 3,1.000000,#mainbody > div.global-body-content-container....,click[383],https://boardgamegeek.com/boardgame/295770/fro...,"[{'url_included_match': 'boardgamegeek.'}, {'u...",3.0,1.000000,1.000000,1.000000
1,100,Find Kevin Durant's bio in espn,finished,3,3 / 3,1.000000,,None,https://www.espn.com/nba/player/bio/_/id/3202/...,"[{'url_included_match': 'espn.'}, {'url_includ...",3.0,1.000000,1.000000,1.000000
2,101,Find camping tents that can fit 6 people and s...,llm_finished,7,3 / 4,0.750000,#search-page-wrapper > div.uhCxDib4luSIS7EWFm8...,"select_option[1137,Price: Low - High]",https://www.rei.com/c/tents/f/sc-6-person?sort...,"[{'url_included_match': 'rei.'}, {'url_exactly...",3.0,0.428571,0.750000,0.750000
3,102,Order an adult’s bottomless bubbles package fo...,limit,16,2 / 8,0.250000,#main > section > div > div > div.col-lg-8 > s...,click[17],https://www.carnival.com/shop/checkout/multi/r...,"[{'url_included_match': 'carnival.'}, {'url_in...",2.0,0.125000,0.200000,0.200000
4,103,"Find Airport information of Camarillo Airport,...",llm_finished,6,1 / 3,0.333333,,None,https://www.flightaware.com/,[{'url_included_match': 'flightaware.'}],1.0,0.166667,0.333333,0.333333


In [16]:
df_evaluate = df[["task_name","status","steps","task_score","task_score_rate","step_score","efficitent_score","human_alignment_score"]]

In [17]:
df_evaluate.head(10)

,task_name,status,steps,task_score,task_score_rate,step_score,efficitent_score,human_alignment_score
0,View the complexity rating for Frosthaven in b...,finished,3,3 / 3,1.000000,3.0,1.000000,1.000000
1,Find Kevin Durant's bio in espn,finished,3,3 / 3,1.000000,3.0,1.000000,1.000000
2,Find camping tents that can fit 6 people and s...,llm_finished,7,3 / 4,0.750000,3.0,0.428571,0.750000
3,Order an adult’s bottomless bubbles package fo...,limit,16,2 / 8,0.250000,2.0,0.125000,0.200000
4,"Find Airport information of Camarillo Airport,...",llm_finished,6,1 / 3,0.333333,1.0,0.166667,0.333333
5,Give a like to the #1 track of the Real Time T...,llm_finished,4,2 / 4,0.500000,2.0,0.500000,0.500000
6,Find the newest silver-colored Lexus SUV with ...,limit,25,1 / 20,0.050000,1.0,0.040000,0.040000
7,Find the score of the 2020 Super Bowl in nfl.com,llm_finished,4,2 / 4,0.500000,2.0,0.500000,0.500000
8,Check my trip for Smith and booking number X89...,limit,12,1 / 4,0.250000,1.0,0.083333,0.200000
9,Show me the stats of the Athletic Club from sp...,llm_finished,3,4 / 5,0.800000,4.0,1.333333,0.800000


In [18]:
df["human_alianment_score"].mean()

0.4256040903540903

In [19]:
df["efficitent_score"].mean()

0.32294663433732257

In [44]:
def evaluate(out_file_path):
    def read_csv_result(file_path=out_file_path):
        with open(file_path) as f:
            data = json.load(f)
        last_action_result_list = []
        for items in data:
            data_dic = {}
            data_dic["task_id"] = items["task_id"]
            data_dic["task_name"] = items["task_name"]
            data_dic["status"] = items["task_status"]
            data_dic["steps"] = items["step_list"][-1]["step_index"]
            data_dic["task_score"] = items["step_list"][-1]["task_score"]
            data_dic["task_score_rate"] = items["step_list"][-1]["task_score_rate"]
            data_dic["selector"] = items["step_list"][-1]["selector"]
            data_dic["action"] = items["step_list"][-1]["action"]
            data_dic["url"] = items["step_list"][-2]["url"]
            # data_dic["match_result"] = items["step_list"][-1]["match_result"]
            last_action_result_list.append(data_dic)
        # result_list = sorted(data,key=lambda x: x["step_list"]["task_score_rate"],reverse=True)
        return last_action_result_list
    all_data = read_csv_result()
    df = pd.DataFrame(all_data)
    df["step_score"] = df["task_score"].apply(lambda x: float(x.split("/")[0]))
    df["efficiency_score"] = df["step_score"] / df["steps"]

    def get_human_alignment_score(row):
        if row["status"] == "finished":
            return 1
        elif row["status"] == "llm_finished":
            return row["task_score_rate"]
        elif row["status"] == "limit":
            return row["task_score_rate"] * 0.8
    df['human_alignment_score'] = df.apply(get_human_alignment_score, axis=1)

    df_evaluate = df[["task_name", "status", "steps", "task_score",
                      "task_score_rate", "step_score", "efficiency_score", "human_alignment_score"]]

    def calculate_total_score(scores):
        molecular_sum = sum(float(x.split('/')[0]) for x in scores)
        denominator_sum = sum(float(x.split('/')[1]) for x in scores)
        final_score = molecular_sum / denominator_sum
        return final_score
    step_score_rate = calculate_total_score(df_evaluate['task_score'])

    completion_rate = df_evaluate[df_evaluate["status"]
                                  == "finished"].shape[0] / df_evaluate.shape[0]

    print(df_evaluate.head(5))
    print(df_evaluate.shape)

    average_step_score_rate = df_evaluate["task_score_rate"].mean()
    average_human_alignment_score = df_evaluate["human_alignment_score"].mean()
    average_efficiency_score = df_evaluate["efficiency_score"].mean()

    print("average_step_score_rate", average_step_score_rate)
    print("average_human_alignment_score", average_human_alignment_score)
    print("average_efficiency_score", average_efficiency_score)
    print("step_score_rate", step_score_rate)
    print("completion_rate", completion_rate)

In [38]:
evaluate(out_file_path="gpt4_planning_gpt4_reward/out.json")

                                           task_name        status  steps  \
0  View the complexity rating for Frosthaven in b...      finished      3   
1                    Find Kevin Durant's bio in espn      finished      3   
2  Find camping tents that can fit 6 people and s...  llm_finished      7   
3  Order an adult’s bottomless bubbles package fo...         limit     16   
4  Find Airport information of Camarillo Airport,...  llm_finished      6   

  task_score  task_score_rate  step_score  efficiency_score  \
0      3 / 3         1.000000         3.0          1.000000   
1      3 / 3         1.000000         3.0          1.000000   
2      3 / 4         0.750000         3.0          0.428571   
3      2 / 8         0.250000         2.0          0.125000   
4      1 / 3         0.333333         1.0          0.166667   

   human_alignment_score  
0               1.000000  
1               1.000000  
2               0.750000  
3               0.200000  
4               0.33333

In [39]:
evaluate(out_file_path="gpt35_planning_gpt4_reward_0416/out.json")

                                           task_name        status  steps  \
0  View the complexity rating for Frosthaven in b...  llm_finished      3   
1  View the full menu for AMC Dine-In in amctheatres  llm_finished      5   
2  Search for used Jaguar XF with no black exteri...         limit     24   
3  Browse for wall art with a price range of $25 ...  llm_finished      6   
4  Search for queen-size pillow protectors from t...         limit     19   

  task_score  task_score_rate  step_score  efficiency_score  \
0      1 / 3         0.333333         1.0          0.333333   
1      3 / 4         0.750000         3.0          0.600000   
2      1 / 7         0.142857         1.0          0.041667   
3      1 / 3         0.333333         1.0          0.166667   
4      2 / 8         0.250000         2.0          0.105263   

   human_alignment_score  
0               0.333333  
1               0.750000  
2               0.114286  
3               0.333333  
4               0.20000

In [40]:
evaluate(out_file_path="planning_gpt4_reward_gpt35/out.json")

                                           task_name    status  steps  \
0  View the complexity rating for Frosthaven in b...  finished      4   
1  View the full menu for AMC Dine-In in amctheatres     limit     10   
2  Search for used Jaguar XF with no black exteri...     limit     24   
3  Browse for wall art with a price range of $25 ...  finished      5   
4  Search for queen-size pillow protectors from t...     limit     19   

  task_score  task_score_rate  step_score  efficiency_score  \
0      3 / 3         1.000000         3.0          0.750000   
1      3 / 4         0.750000         3.0          0.300000   
2      2 / 7         0.285714         2.0          0.083333   
3      3 / 3         1.000000         3.0          0.600000   
4      4 / 8         0.500000         4.0          0.210526   

   human_alignment_score  
0               1.000000  
1               0.600000  
2               0.228571  
3               1.000000  
4               0.400000  
(130, 8)
average_ste

In [41]:
evaluate(out_file_path="planning_only_gpt35/out.json")

                                           task_name    status  steps  \
0  View the complexity rating for Frosthaven in b...  finished      7   
1  View the full menu for AMC Dine-In in amctheatres     limit     10   
2  Search for used Jaguar XF with no black exteri...     limit     24   
3  Browse for wall art with a price range of $25 ...     limit     10   
4  Search for queen-size pillow protectors from t...     limit     19   

  task_score  task_score_rate  step_score  efficiency_score  \
0      3 / 3         1.000000         3.0          0.428571   
1      3 / 4         0.750000         3.0          0.300000   
2      1 / 7         0.142857         1.0          0.041667   
3      1 / 3         0.333333         1.0          0.100000   
4      3 / 8         0.375000         3.0          0.157895   

   human_alignment_score  
0               1.000000  
1               0.600000  
2               0.114286  
3               0.266667  
4               0.300000  
(130, 8)
average_ste

In [42]:
evaluate(out_file_path="planning_only_0402/out.json")

                                           task_name    status  steps  \
0  Check my trip for Smith and booking number X89...     limit     12   
1  Show me the stats of the Athletic Club from sp...     limit     10   
2  Search for video game consoles and filter the ...  finished      5   
3  Find a Ricky Kej track to listen and share whi...     limit     18   
4  Browse and purchase the classic bundle for the...     limit     15   

  task_score  task_score_rate  step_score  efficiency_score  \
0      1 / 4             0.25         1.0          0.083333   
1      4 / 5             0.80         4.0          0.400000   
2      3 / 3             1.00         3.0          0.600000   
3      2 / 5             0.40         2.0          0.111111   
4      1 / 4             0.25         1.0          0.066667   

   human_alignment_score  
0                   0.20  
1                   0.64  
2                   1.00  
3                   0.32  
4                   0.20  
(130, 8)
average_ste

In [2]:
def evaluate(out_file_path):
    def read_csv_result(file_path=out_file_path):
        with open(file_path) as f:
            data = json.load(f)
        last_action_result_list = []
        for items in data:
            data_dic = {}
            data_dic["task_id"] = items["task_id"]
            data_dic["task_name"] = items["task_name"]
            data_dic["status"] = items["task_status"]
            data_dic["steps"] = items["step_list"][-1]["step_index"]
            data_dic["task_score"] = items["step_list"][-1]["task_score"]
            data_dic["task_score_rate"] = items["step_list"][-1]["task_score_rate"]
            data_dic["selector"] = items["step_list"][-1]["selector"]
            data_dic["action"] = items["step_list"][-1]["action"]
            data_dic["url"] = items["step_list"][-2]["url"]
            # data_dic["match_result"] = items["step_list"][-1]["match_result"]
            last_action_result_list.append(data_dic)
        # result_list = sorted(data,key=lambda x: x["step_list"]["task_score_rate"],reverse=True)
        return last_action_result_list
    all_data = read_csv_result()
    df = pd.DataFrame(all_data)
    df["step_score"] = df["task_score"].apply(lambda x: float(x.split("/")[0]))
    df["efficiency_score"] = df["step_score"] / df["steps"]

    def get_llm_finished(row):
        if row["status"] == "finished" and row["task_score_rate"] < 1.0:
            return "llm_finished"
        else:
            return row["status"]
        
    df["status"] = df.apply(get_llm_finished,axis=1)
    
    def get_human_alignment_score(row):
        if row["status"] == "finished":
            return 1
        elif row["status"] == "llm_finished":
            return row["task_score_rate"]
        elif row["status"] == "limit":
            return row["task_score_rate"] * 0.8
        
    df['human_alignment_score'] = df.apply(get_human_alignment_score, axis=1)

    df_evaluate = df[["task_name","status","steps","task_score","task_score_rate","step_score","efficiency_score","human_alignment_score"]]

    def calculate_total_score(scores):
        molecular_sum = sum(float(x.split('/')[0]) for x in scores)
        denominator_sum = sum(float(x.split('/')[1]) for x in scores)
        final_score = molecular_sum / denominator_sum
        return final_score
    step_score_rate = calculate_total_score(df_evaluate['task_score'])

    completion_rate = df_evaluate[df_evaluate["status"] == "finished"].shape[0] / df_evaluate.shape[0]

    print(df_evaluate.head(5))
    print(df_evaluate.shape)

    average_step_score_rate = df_evaluate["task_score_rate"].mean()
    average_human_alignment_score = df_evaluate["human_alignment_score"].mean()
    average_efficiency_score = df_evaluate["efficiency_score"].mean()

    print("average_step_score_rate",average_step_score_rate)
    print("average_human_alignment_score",average_human_alignment_score)
    print("average_efficiency_score",average_efficiency_score)
    print("step_score_rate",step_score_rate)
    print("completion_rate",completion_rate)




In [3]:
evaluate(out_file_path="planning_only_gpt4_without_thought/out.json")

                                           task_name status  steps task_score  \
0  View the complexity rating for Frosthaven in b...  limit     10      2 / 3   
1  View the full menu for AMC Dine-In in amctheatres  limit     10      3 / 4   
2  Search for used Jaguar XF with no black exteri...  limit     24      1 / 7   
3  Browse for wall art with a price range of $25 ...  limit     10   2.75 / 3   
4  Search for queen-size pillow protectors from t...  limit     19      4 / 8   

   task_score_rate  step_score  efficiency_score  human_alignment_score  
0         0.666667        2.00          0.200000               0.533333  
1         0.750000        3.00          0.300000               0.600000  
2         0.142857        1.00          0.041667               0.114286  
3         0.916667        2.75          0.275000               0.733333  
4         0.500000        4.00          0.210526               0.400000  
(130, 8)
average_step_score_rate 0.49877899877899873
average_human_al

In [4]:
evaluate(out_file_path="planning_gpt4_without_trace/out.json")

                                           task_name status  steps task_score  \
0  View the complexity rating for Frosthaven in b...  limit     10      2 / 3   
1  View the full menu for AMC Dine-In in amctheatres  limit     10      2 / 4   
2  Search for used Jaguar XF with no black exteri...  limit     24      1 / 7   
3  Browse for wall art with a price range of $25 ...  limit     10      1 / 3   
4  Search for queen-size pillow protectors from t...  limit     19      5 / 8   

   task_score_rate  step_score  efficiency_score  human_alignment_score  
0         0.666667         2.0          0.200000               0.533333  
1         0.500000         2.0          0.200000               0.400000  
2         0.142857         1.0          0.041667               0.114286  
3         0.333333         1.0          0.100000               0.266667  
4         0.625000         5.0          0.263158               0.500000  
(130, 8)
average_step_score_rate 0.4807509157509157
average_human_ali

In [9]:
evaluate(out_file_path="48DOMmode(GPT-4)+DOM&Visionreward(GPT-4V)/out.json")

                                           task_name        status  steps  \
0  Check my trip for Smith and booking number X89...  llm_finished      4   
1  Show me the stats of the Athletic Club from sp...  llm_finished      3   
2  Search for video game consoles and filter the ...  llm_finished      3   
3  Find a Ricky Kej track to listen and share whi...  llm_finished     12   
4  Browse and purchase the classic bundle for the...         limit     15   

  task_score  task_score_rate  step_score  efficiency_score  \
0      2 / 4         0.500000         2.0          0.500000   
1      4 / 5         0.800000         4.0          1.333333   
2      1 / 3         0.333333         1.0          0.333333   
3      2 / 5         0.400000         2.0          0.166667   
4      1 / 4         0.250000         1.0          0.066667   

   human_alignment_score  
0               0.500000  
1               0.800000  
2               0.333333  
3               0.400000  
4               0.20000